In [1]:
import pandas as pd
import numpy as np

In [2]:
pre_final_lab=pd.read_csv("labs_after_FiO2.csv")
susp=pd.read_csv("24_hour_sepsis.csv")

In [ ]:
pre_final_lab.head()

In [4]:
pre_final_lab['platelets_x_1000']=np.nan
pre_final_lab['total_bilirubin']=np.nan
pre_final_lab['urinary_creatinine']=np.nan

In [5]:
pre_final_lab['platelets_x_1000']=pre_final_lab.loc[pre_final_lab['labname']=='platelets x 1000','labresult']
pre_final_lab['total_bilirubin']=pre_final_lab.loc[pre_final_lab['labname']=='total bilirubin','labresult']
pre_final_lab['urinary_creatinine']=pre_final_lab.loc[pre_final_lab['labname']=='urinary creatinine','labresult']

In [ ]:
pre_final_lab

In [7]:
pre_final_lab['paO2_FiO2']=pre_final_lab['paO2_FiO2'].astype('float32')
pre_final_lab['platelets_x_1000']=pre_final_lab['platelets_x_1000'].astype('float32')
pre_final_lab['total_bilirubin']=pre_final_lab['total_bilirubin'].astype('float32')
pre_final_lab['urinary_creatinine']=pre_final_lab['urinary_creatinine'].astype('float32')

In [8]:
pre_final_lab.dtypes

patientunitstayid       int64
labresultoffset         int64
labname                object
labresult             float64
paO2_FiO2             float32
platelets_x_1000      float32
total_bilirubin       float32
urinary_creatinine    float32
dtype: object

In [9]:
pre_final_lab=pre_final_lab.replace([np.inf, -np.inf], np.nan)

In [10]:
del pre_final_lab['labresult']

In [11]:
del pre_final_lab['labname']

In [ ]:
pre_final_lab

In [15]:
cases=list(susp[susp['flag']==1]['patientunitstayid'].unique())
filtered_prefinal=pre_final_lab.loc[pre_final_lab['patientunitstayid'].isin(cases)]
len(list(filtered_prefinal['patientunitstayid'].unique()))

14156

In [19]:
df_lab=pd.read_csv('lab.csv',chunksize=10000)
arr=[]
for chunk in df_lab:
    arr.append(chunk)
df_lab=pd.concat(arr,sort=False)
creatinine_to_merge=df_lab
creatinine_to_merge['creatinine']=np.nan
creatinine_to_merge.loc[creatinine_to_merge['labname'].str.startswith('creatinine'),'creatinine']=creatinine_to_merge['labresult']
creatinine_to_merge=creatinine_to_merge.loc[creatinine_to_merge['labname'].str.startswith('creatinine')]
creatinine_to_merge=creatinine_to_merge[['patientunitstayid','labresultoffset','creatinine']]
creatinine_to_merge.to_csv("creatinine_extract.csv",index=False)

In [16]:
creatinine_to_merge=pd.read_csv('creatinine_extract.csv')

In [17]:
len(creatinine_to_merge.loc[creatinine_to_merge['patientunitstayid'].isin(cases)]['patientunitstayid'].unique())

14151

In [21]:
labswithcreatinine=pd.merge(pre_final_lab,creatinine_to_merge,on=['patientunitstayid','labresultoffset'],how='outer')
labswithcreatinine=labswithcreatinine.groupby(['patientunitstayid','labresultoffset'],as_index=False).max()

In [22]:
labswithcreatinine.to_csv("labs_updated_with_creatinine.csv",index=False)

In [16]:
labswithcreatinine.head()

patientunitstayid  labresultoffset  paO2_FiO2  platelets_x_1000  \
0             141168              516        NaN             209.0   
1             141168             1133        NaN             213.0   
2             141168             1701        NaN               NaN   
3             141168             1805       82.0               NaN   
4             141168             2010       84.0               NaN   

   total_bilirubin  urinary_creatinine  creatinine  
0              2.6                 NaN        1.95  
1              4.1                 NaN        2.30  
2              NaN          173.119995         NaN  
3              NaN                 NaN         NaN  
4              NaN                 NaN         NaN

In [17]:
nursevent=pd.read_csv("nursechartVent.csv")

In [ ]:
nursevent.head()

In [19]:
nursevent=nursevent[['patientunitstayid','nursingchartentryoffset','nursingchartvalue']]

In [36]:
#now we will merge ventilator support with labs table
#we need all the labs columns, but only need info about ventilator support if available, so we will use a left join instead of an outer join
labs_withO2=pd.merge(labswithcreatinine,nursevent,left_on=['patientunitstayid'],right_on=['patientunitstayid'],how='left').drop_duplicates()

In [ ]:
#This is fairly how we want the table to look like, but the nursingchartentryoffset values need to filtered before using for paO2_FiO2
labs_withO2.loc[(labs_withO2['patientunitstayid']==144297)]


In [38]:
labs_withO2.head()

patientunitstayid  labresultoffset  paO2_FiO2  platelets_x_1000  \
0             141168              516        NaN             209.0   
1             141168             1133        NaN             213.0   
2             141168             1701        NaN               NaN   
3             141168             1805       82.0               NaN   
4             141168             2010       84.0               NaN   

   total_bilirubin  urinary_creatinine  creatinine  nursingchartentryoffset  \
0              2.6                 NaN        1.95                    513.0   
1              4.1                 NaN        2.30                    513.0   
2              NaN          173.119995         NaN                    513.0   
3              NaN                 NaN         NaN                    513.0   
4              NaN                 NaN         NaN                    513.0   

  nursingchartvalue  
0     nasal cannula  
1     nasal cannula  
2     nasal cannula  
3     nasal cannula  
4     nasal cannula

In [39]:
#Include SOFA scores

labs_withO2.loc[(labs_withO2['platelets_x_1000'] >=150), 'SOFA_Coagulation'] = 0
labs_withO2.loc[(labs_withO2['platelets_x_1000'] <150), 'SOFA_Coagulation'] = 1
labs_withO2.loc[(labs_withO2['platelets_x_1000'] <100) , 'SOFA_Coagulation'] = 2
labs_withO2.loc[(labs_withO2['platelets_x_1000'] <50), 'SOFA_Coagulation'] = 3
labs_withO2.loc[(labs_withO2['platelets_x_1000'] <20), 'SOFA_Coagulation'] = 4

labs_withO2.loc[(labs_withO2['total_bilirubin'] <1.2), 'SOFA_Liver'] = 0
labs_withO2.loc[(labs_withO2['total_bilirubin'] >=1.2) & (labs_withO2['total_bilirubin'] <=1.9), 'SOFA_Liver'] = 1
labs_withO2.loc[(labs_withO2['total_bilirubin'] >=2) & (labs_withO2['total_bilirubin'] <=5.9), 'SOFA_Liver'] = 2
labs_withO2.loc[(labs_withO2['total_bilirubin'] >=6) & (labs_withO2['total_bilirubin'] <=11.9), 'SOFA_Liver'] = 3
labs_withO2.loc[(labs_withO2['total_bilirubin'] >12), 'SOFA_Liver'] = 4

labs_withO2.loc[(labs_withO2['paO2_FiO2'] >=400), 'SOFA_Respiration'] = 0
labs_withO2.loc[(labs_withO2['paO2_FiO2'] <400), 'SOFA_Respiration'] = 1
labs_withO2.loc[(labs_withO2['paO2_FiO2'] <300), 'SOFA_Respiration'] = 2
labs_withO2.loc[((labs_withO2['paO2_FiO2'] <200) & (labs_withO2['nursingchartvalue'] =='ventilator')), 'SOFA_Respiration'] = 3
labs_withO2.loc[((labs_withO2['paO2_FiO2'] <100) & (labs_withO2['nursingchartvalue'] =='ventilator')), 'SOFA_Respiration'] = 4

labs_withO2.loc[((labs_withO2['creatinine'] >=0) & (labs_withO2['creatinine'] <=1.1)), 'SOFA_Renal'] = 0
labs_withO2.loc[((labs_withO2['creatinine'] >=1.2) & (labs_withO2['creatinine'] <=1.9)), 'SOFA_Renal'] = 1
labs_withO2.loc[((labs_withO2['creatinine'] >=2) & (labs_withO2['creatinine'] <=3.4)), 'SOFA_Renal'] = 2
labs_withO2.loc[((labs_withO2['creatinine'] >=3.5) & (labs_withO2['creatinine'] <=4.9)) | (labs_withO2['urinary_creatinine'] <200), 'SOFA_Renal'] = 3
labs_withO2.loc[(labs_withO2['creatinine'] >5) | (labs_withO2['urinary_creatinine'] <200), 'SOFA_Renal'] = 4



In [40]:
labs_withO2.head()

patientunitstayid  labresultoffset  paO2_FiO2  platelets_x_1000  \
0             141168              516        NaN             209.0   
1             141168             1133        NaN             213.0   
2             141168             1701        NaN               NaN   
3             141168             1805       82.0               NaN   
4             141168             2010       84.0               NaN   

   total_bilirubin  urinary_creatinine  creatinine  nursingchartentryoffset  \
0              2.6                 NaN        1.95                    513.0   
1              4.1                 NaN        2.30                    513.0   
2              NaN          173.119995         NaN                    513.0   
3              NaN                 NaN         NaN                    513.0   
4              NaN                 NaN         NaN                    513.0   

  nursingchartvalue  SOFA_Coagulation  SOFA_Liver  SOFA_Respiration  \
0     nasal cannula               0.0         2.0               NaN   
1     nasal cannula               0.0         2.0               NaN   
2     nasal cannula               NaN         NaN               NaN   
3     nasal cannula               NaN         NaN               2.0   
4     nasal cannula               NaN         NaN               2.0   

   SOFA_Renal  
0         NaN  
1         2.0  
2         4.0  
3         NaN  
4         NaN

In [41]:
labs_withO2['offset'] = np.where(labs_withO2['labresultoffset']>0, labs_withO2['labresultoffset'], labs_withO2['nursingchartentryoffset'])
labs_withO2[['SOFA_Coagulation','SOFA_Liver','SOFA_Respiration','SOFA_Renal']]=labs_withO2[['SOFA_Coagulation','SOFA_Liver','SOFA_Respiration','SOFA_Renal']].fillna(0)
labs_withO2_filtered=labs_withO2[['patientunitstayid','offset','SOFA_Coagulation','SOFA_Liver','SOFA_Respiration','SOFA_Renal']].drop_duplicates()
labs_withO2_filtered.to_csv('Labs_withSOFA.csv',index=False)

In [ ]:
labs_withO2_filtered

<h1>Can be done</h1>
pre_final_lab['paO2_FiO2']=pre_final_lab['paO2_FiO2'].ffill(limit=5)
pre_final_lab['platelets_x_1000']=pre_final_lab['platelets_x_1000'].ffill(limit=5)
pre_final_lab['total_bilirubin']=pre_final_lab['total_bilirubin'].ffill(limit=5)
pre_final_lab['urinary_creatinine']=pre_final_lab['urinary_creatinine'].ffill(limit=5)
pre_final_lab['paO2_FiO2']=pre_final_lab['paO2_FiO2'].bfill(limit=5)
pre_final_lab['platelets_x_1000']=pre_final_lab['platelets_x_1000'].bfill(limit=5)
pre_final_lab['total_bilirubin']=pre_final_lab['total_bilirubin'].bfill(limit=5)
pre_final_lab['urinary_creatinine']=pre_final_lab['urinary_creatinine'].bfill(limit=5)



In [13]:
del pre_final_lab['labname']

In [ ]:
pre_final_lab.head()

In [15]:
pre_final_lab.to_csv('labs_updated.csv',sep=',',index=False,encoding='utf-8')

<h4>Adding additional parameters from physionet challenge data</h4>

In [3]:
df_lab=pd.read_csv('lab.csv',chunksize=100000)

In [4]:
arr=[]
for chunk in df_lab:
    arr.append(chunk)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df_lab=pd.concat(arr,sort=False)

In [21]:
df_to_join=df_lab.loc[(df_lab['labname'].str.contains('HCO3')) | 
           (df_lab['labname'].str.contains('pH')) |
           (df_lab['labname'].str.contains('paCO2',case=False)) |
           (df_lab['labname'].str.contains('direct',case=False)) |
           (df_lab['labname'].str.contains('excess',case=False)) |
           (df_lab['labname'].str.contains('ast',case=False)) |
           (df_lab['labname'].str.contains('bun',case=False)) |
           (df_lab['labname'].str.contains('Calcium',case=False)) |
           (df_lab['labname'].str.contains('glucose',case=False)) |
           (df_lab['labname'].str.contains('lactate',case=False)) |
           (df_lab['labname'].str.contains('magnesium',case=False)) |
           (df_lab['labname'].str.contains('phosphate',case=False)) |
           (df_lab['labname'].str.contains('potassium',case=False)) |
           (df_lab['labname'].str.contains('hct',case=False)) |
           (df_lab['labname'].str.contains('hgb',case=False)) |
           (df_lab['labname'].str.contains('ptt',case=False)) |
           (df_lab['labname'].str.contains('WBC x 1000')) |
           (df_lab['labname'].str.contains('fibrinogen',case=False)) |
           (df_lab['labname'].str.contains('troponin',case=False))]

In [22]:
df_to_join=df_to_join[['patientunitstayid','labresultoffset','labname','labresult']]

In [23]:
df_to_join.loc[df_to_join['labname'].str.contains('HCO3'),'HCO3']=df_to_join['labresult']
df_to_join.loc[df_to_join['labname'].str.contains('pH'),'pH']=df_to_join['labresult']
df_to_join.loc[df_to_join['labname'].str.contains('paCO2',case=False),'paCO2']=df_to_join['labresult']
df_to_join.loc[df_to_join['labname'].str.contains('direct',case=False),'direct_bilirubin']=df_to_join['labresult']
df_to_join.loc[df_to_join['labname'].str.contains('excess',case=False),'excess']=df_to_join['labresult']
df_to_join.loc[df_to_join['labname'].str.contains('ast',case=False),'ast']=df_to_join['labresult']
df_to_join.loc[df_to_join['labname'].str.contains('bun',case=False),'bun']=df_to_join['labresult']
df_to_join.loc[df_to_join['labname'].str.contains('Calcium',case=False),'calcium']=df_to_join['labresult']
df_to_join.loc[df_to_join['labname'].str.contains('glucose',case=False),'glucose']=df_to_join['labresult']
df_to_join.loc[df_to_join['labname'].str.contains('lactate',case=False),'lactate']=df_to_join['labresult']
df_to_join.loc[df_to_join['labname'].str.contains('magnesium',case=False),'magnesium']=df_to_join['labresult']
df_to_join.loc[df_to_join['labname'].str.contains('phosphate',case=False),'phosphate']=df_to_join['labresult']
df_to_join.loc[df_to_join['labname'].str.contains('potassium',case=False),'potassium']=df_to_join['labresult']
df_to_join.loc[df_to_join['labname'].str.contains('hct',case=False),'hct']=df_to_join['labresult']
df_to_join.loc[df_to_join['labname'].str.contains('hgb',case=False),'hgb']=df_to_join['labresult']
df_to_join.loc[df_to_join['labname'].str.contains('ptt',case=False),'ptt']=df_to_join['labresult']
df_to_join.loc[df_to_join['labname'].str.contains('WBC x 1000'),'wbc']=df_to_join['labresult']
df_to_join.loc[df_to_join['labname'].str.contains('fibrinogen',case=False),'fibrinogen']=df_to_join['labresult']
df_to_join.loc[df_to_join['labname'].str.contains('troponin',case=False),'troponin']=df_to_join['labresult']

In [24]:
del df_lab

In [25]:
df_to_join=df_to_join.drop(columns=['labname','labresult'])

In [26]:
#import the labs_updated.csv to join
labs_to_join=pd.read_csv("labs_updated_with_creatinine.csv")

In [27]:
labs_new=pd.merge(labs_to_join,df_to_join,on=['patientunitstayid','labresultoffset'],how='outer')
labs_new=labs_new.drop_duplicates()
labs_new=labs_new.groupby(['patientunitstayid','labresultoffset'],as_index=False).max()

In [ ]:
mrevars=pd.read_csv("labs_morevars.csv")


In [20]:
len(mrevars.loc[mrevars['patientunitstayid'].isin(cases)]['patientunitstayid'].unique())

14175